In [1]:
import gdown
from pathlib import Path
import subprocess
import random

random.seed(0xC0FFEE)

In [2]:
MSBFS_SS = [random.randint(0, 100_000) for _ in range(100)]
SSBFS_S = MSBFS_SS[0]

print(MSBFS_SS)
print(SSBFS_S)

[76317, 81110, 22722, 2139, 15873, 86997, 56185, 24991, 38945, 61652, 53289, 55552, 83158, 96266, 43244, 84808, 21186, 99991, 55457, 83284, 56913, 40048, 88948, 12959, 45343, 14998, 44989, 20071, 32542, 56292, 9144, 17412, 69825, 87261, 26158, 49386, 72275, 62558, 57220, 24471, 23493, 94994, 37980, 4091, 9009, 66525, 33972, 29836, 52672, 87383, 26338, 13704, 36551, 44173, 715, 66505, 5876, 83236, 8911, 13990, 31235, 43621, 27414, 79798, 13544, 72640, 17165, 42792, 15747, 34318, 15090, 34718, 27247, 13394, 51563, 60557, 90254, 86298, 1979, 42284, 85643, 57489, 68188, 50753, 60441, 65614, 40101, 70979, 21118, 34973, 42012, 90485, 62079, 73485, 62898, 14564, 5767, 25093, 72883, 56183]
76317


In [3]:
PROJECT_DIR = Path(".").resolve().parent

In [4]:
DATASETS_DIR = PROJECT_DIR / "datasets"
DIMACS_DIR = DATASETS_DIR / "dimacs"
SNAP_DIR = DATASETS_DIR / "snap"

In [ ]:
if not DATASETS_DIR.exists():
    gdown.download_folder(
        url="https://drive.google.com/drive/folders/1_l20PoxHFo0VZN1QVwZ5eRTyqenlpwlC",
        output=str(DATASETS_DIR),
    )

In [21]:
DIMACS_GRAPHS = {
    "USA": DIMACS_DIR / "USA.gr",
    "NY": DIMACS_DIR / "NY.gr",
    "CAL": DIMACS_DIR / "CAL.gr",
    "FLA": DIMACS_DIR / "FLA.gr",
    "W": DIMACS_DIR / "W.gr",
}

In [22]:
SNAP_GRAPHS = {
    "sx-stackoverflow": SNAP_DIR / "sx-stackoverflow.gr",
    "web-berkstan": SNAP_DIR / "web-berkstan.gr",
    "web-google": SNAP_DIR / "web-google.gr",
    "web-stanford": SNAP_DIR / "web-stanford.gr",
    "wiki-topcats": SNAP_DIR / "wiki-topcats.gr",
}

In [23]:
GB_EXES_DIR = PROJECT_DIR / "build" / "cpp" / "graphblas"
SPLA_EXES_DIR = PROJECT_DIR / "build" / "cpp" / "spla"

ALGOS_EXES = {
    "gb-ssbfs": GB_EXES_DIR / "ssbfs" / "gb-ssbfs",
    "spla-ssbfs": SPLA_EXES_DIR / "ssbfs" / "spla-ssbfs",
    "gb-msbfs": GB_EXES_DIR / "msbfs" / "gb-msbfs",
    "spla-msbfs": SPLA_EXES_DIR / "msbfs" / "spla-msbfs",
    "gb-pagerank": GB_EXES_DIR / "pagerank" / "gb-pagerank",
    "spla-pagerank": SPLA_EXES_DIR / "pagerank" / "spla-pagerank",
    "gb-mst": GB_EXES_DIR / "mst" / "gb-mst",
    "spla-mst": SPLA_EXES_DIR / "mst" / "spla-mst",
}

In [36]:
N_RUNS = 10

In [25]:
RESULTS_DIR = PROJECT_DIR / "results"
RESULTS_DIR.mkdir(exist_ok=True)

In [26]:
def run_algo(exe_path: str, config: dict[str, str]) -> list[int]:
    with open("exp_config.ini", "w") as f:
        for k, v in config.items():
            f.write(f"{k}={v}\n")

    times = []
    for _ in range(N_RUNS):
        exe_args = [exe_path, "exp_config.ini"]
            
        res = subprocess.run(
            args=exe_args, capture_output=True, text=True
        )
        times.append(int(res.stdout))

    return times

In [27]:
RESULTS_DIR = PROJECT_DIR / "results"

SSBFS_RESULTS_DIR = RESULTS_DIR / "ssbfs"
MSBFS_RESULTS_DIR = RESULTS_DIR / "msbfs"
MST_RESULTS_DIR = RESULTS_DIR / "mst"
PAGERANK_RESULTS_DIR = RESULTS_DIR / "pagerank"

In [28]:
def run_ssbfs():
    exes = ["gb-ssbfs", "spla-ssbfs"]
    for exe in exes:
        for graph_name, graph_path in DIMACS_GRAPHS.items():
            print(f"Algo: {exe}, Graph: {graph_name}")
            config = {"GraphPath": graph_path, "StartingVertex": SSBFS_S}
            times = run_algo(ALGOS_EXES[exe], config)

            print("Saving to")
            print(SSBFS_RESULTS_DIR / f"{exe}_{graph_name}.txt")
            with open(
                str(SSBFS_RESULTS_DIR / f"{exe}_{graph_name}.txt"),
                "w",
            ) as f:
                for time in times:
                    f.write(f"{time}\n")

In [29]:
def run_msbfs(n_vertices=1):
    exes = ["spla-msbfs", "gb-msbfs"]
    starting_vertices = list(map(str, MSBFS_SS[:n_vertices]))
    for exe in exes:
        for graph_name, graph_path in DIMACS_GRAPHS.items():
            if graph_name in ["USA", "W"]:
                continue
            
            print(f"Algo: {exe}, Graph: {graph_name}, N={len(starting_vertices)}")
            config = {
                "GraphPath": graph_path,
                "StartingVertices": ",".join(starting_vertices),
            }
            times = run_algo(ALGOS_EXES[exe], config)

            print("Saving to")
            print(MSBFS_RESULTS_DIR / f"{exe}_{graph_name}_N={len(starting_vertices)}.txt")
            with open(
                str(MSBFS_RESULTS_DIR / f"{exe}_{graph_name}_N={len(starting_vertices)}.txt"),
                "w",
            ) as f:
                for time in times:
                    f.write(f"{time}\n")

In [39]:
def run_pagerank():
    exes = ["gb-pagerank", "spla-pagerank"]
    for exe in exes:
        for graph_name, graph_path in SNAP_GRAPHS.items():
            if 'gb' in exe:
                continue
            print(f"Algo: {exe}, Graph: {graph_name}")
            config = {"GraphPath": graph_path}
            times = run_algo(ALGOS_EXES[exe], config)

            print("Saving to")
            print(PAGERANK_RESULTS_DIR / f"{exe}_{graph_name}.txt")
            with open(
                str(PAGERANK_RESULTS_DIR / f"{exe}_{graph_name}.txt"),
                "w",
            ) as f:
                for time in times:
                    f.write(f"{time}\n")

In [31]:
def run_mst():
    exes = ["gb-mst", "spla-mst"]
    for exe in exes:
        for graph_name, graph_path in DIMACS_GRAPHS.items():
            print(f"Algo: {exe}, Graph: {graph_name}")
            config = {"GraphPath": graph_path}
            times = run_algo(ALGOS_EXES[exe], config)

            print("Saving to")
            print(MST_RESULTS_DIR / f"{exe}_{graph_name}.txt")
            with open(
                str(MST_RESULTS_DIR / f"{exe}_{graph_name}.txt"),
                "w",
            ) as f:
                for time in times:
                    f.write(f"{time}\n")

In [32]:
if not SSBFS_RESULTS_DIR.exists():
    SSBFS_RESULTS_DIR.mkdir()
    run_ssbfs()

In [33]:
if not MSBFS_RESULTS_DIR.exists():
    MSBFS_RESULTS_DIR.mkdir()
    run_msbfs(4)
    run_msbfs(1)
    run_msbfs(16)
    run_msbfs(64)

In [34]:
if not MST_RESULTS_DIR.exists():
    MST_RESULTS_DIR.mkdir()
    run_mst()

In [ ]:
if not PAGERANK_RESULTS_DIR.exists():
    PAGERANK_RESULTS_DIR.mkdir()
    run_pagerank()